<a href="https://colab.research.google.com/github/yecatstevir/teambrainiac/blob/main/source/SingleSubjectSVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Whole Brain Support Vector Machine Training
- Go to 'Runtime' in Colab browser bar, select 'Change Runtime Type', select 'High-RAM' from 'Runtime Shape'. 
- load local pickle file containing all masked, normalized Whole Brain subject data in numpy matrix format
- SVM training per subject

### Mount Google Drive and clone repository
- open to source directory

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')#, force_remount = True)

Mounted at /content/gdrive


In [2]:

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
# Clone the entire repo.
!git clone -l -s https://github.com/yecatstevir/teambrainiac.git
# Change directory into cloned repo
%cd teambrainiac/source
!ls


Cloning into 'teambrainiac'...
remote: Enumerating objects: 745, done.
remote: Counting objects: 100% (745/745), done.
remote: Compressing objects: 100% (542/542), done.
remote: Total 745 (delta 462), reused 380 (delta 187), pack-reused 0
Receiving objects: 100% (745/745), 71.83 MiB | 32.29 MiB/s, done.
Resolving deltas: 100% (462/462), done.
/content/teambrainiac/source
access_data.py		  process.py
AccuracyMeasures.ipynb	  SingleSubjectSVM.ipynb
analysis.py		  SubjectVisualization_Models_ZNORM.ipynb
cross_validation.py	  SVM_Group_Adolescent_Whole_brain.ipynb
data			  SVM_Group_YA_Whole_brain.ipynb
DL			  TestMask.ipynb
Explore_data.ipynb	  train.py
Group_All_MASK_SVM.ipynb  utils.py
helper			  VisualizationPlayground.ipynb
Images			  Visualize_Data.ipynb
__init__.py		  visualize.py
models


### Load path_config.py 
- we are already in source so we can just load this file without changing directory

In [5]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving path_config.py to path_config.py
User uploaded file "path_config.py" with length 228 bytes


### Import libraries


In [6]:

# Import libraries
!pip install boto3 #for saving data
import pickle
#sklearn packages needed
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

#important utility functions for loading,masking,saving data
from utils import *
#normal python packages we use
import numpy as np
import pandas as pd


### Get paths to subject data and grab labels for SVM

In [7]:

## load and open the pickle file that contains paths to all data.
path = "data/data_path_dictionary.pkl"
data_path_dict = open_pickle(path)

Functions to get information about data to run our SVM

In [ ]:
def get_subj_information(data_path_dict):
  """
    Function to get subject information.
    data_path_dict  : dictionary containing paths to all data stored on AWS
    returns:  subject_ids(list of subjects to run),subj_paths(paths to subject raw data)
  """
  subject_ids = data_path_dict['subject_ID'] #subject_ids
  subj_paths = data_path_dict['subject_data'] #subject_paths
  return subject_ids,subj_paths

def get_labels(data_path_dict):
  """
    Function to get the labels for our data.
    data_path_dict  : dictionary containing paths to all data stored on AWS
    returns: mask_labels_indices(timepoints we want masked out),binary_labels(labels for our for our two brain states)
  """
  
  label_data_path = data_path_dict['labels'][0] #get labels
  label_type = 'rt_labels' #tell the function what labels we want
  mask_labels_indices, binary_labels = labels_mask_binary(label_data_path, label_type) #grab indices and labels
  return mask_labels_indices, binary_labels

def get_mask(mask_type,data_path_dict,mask_ind):
  """
    Function to return the mask of what brain voxels we want to include in analysis
    data_path_dict  : dictionary containing paths to all data stored on AWS
    mask_type: name of mask we want to use
    mask_ind: index of where the path to the masks are 0: full brain mask plus masks that subtract region
              1: Regions of interest(ROIs) mask out full brain except structure we care about
  """
  mask_data_filepath = data_path_dict['mask_data'][mask_ind] #path to masked data     
  mask_type_dict = access_load_data(mask_data_filepath, True) #get the mask data dictionary
  np_array_mask = mask_type_dict[mask_type] #get the mask array
  mask = np.ma.make_mask(np_array_mask).reshape(79*95*79,order='F') #create a 1-D array for the mask. Important to use Fourier Transformation as we are working in brain space!

  return mask

In [ ]:
mask_data_filepath = data_path_dict['mask_data'][0] #path to masked data  
mask_type_dict = access_load_data(mask_data_filepath, True)
mask_type_dict.keys()

dict_keys(['__header__', '__version__', '__globals__', 'mask', 'masksubACC', 'masksubAI', 'masksubNAcc', 'masksubmPFC'])

## Set up SVM Model

In [ ]:
def load_subject_data(subj_paths,subject_ids,idx,dopsc,mask,mask_labels_indices,binary_labels):
  """
    Function to load indivdual subject data. Calls masking data which is in utils.py
      subj_paths  :paths to subject data
      subject_ids :list of subjects
      idx   : index of current subject
      dopsc : If True, do percent signal change before returning user_data_dict
      mask  : mask to use on subject
      mask_labels_indices : indices of brain volumes we want to keep (aligning to increase/decrease)
      binary_labels: labels of increase/decrease to append to user dictionary
      returns: user data dictionary that contains all four runs masked to just keep voxels of interest along with labels
               along with the subjects id
  """
  mat_path = subj_paths[idx] #get raw data path
  sub_id = subject_ids[idx] #get subject id
  data = access_load_data(mat_path,True) #call function to load data and return a dictionary. We are loading .mat files so need to set second param to True
  user_data_dict = {} #create empty dict
  user_data_dict[sub_id], bi_lb = masking_data(data, mask, mask_labels_indices, binary_labels, dopsc) #call function to mask the data and do normalization if desired
  user_data_dict[f"{sub_id}_{label_type}"] = bi_lb #set the binary labels for our data (0 = decrease,1=increase)
  return user_data_dict,sub_id

In [ ]:
def scale_data_single_subj(sub_data,sub_labels, runs_train,runs_test,norm):
  """
    Function to scale data. Flexible to accomadate multiple different schemas to test which normalization is best.
      sub_data     : (1 subject data, keys as subject ID for frmi data or labels)
      sub_labels   : sub_labels to indicate which row of the sub_data belongs to increase/decrease state
      runs_train   : tuple , (which run are we using for the training data)
                            If runs_test = 1, there will be no X_val, y_val
      runs_test    : tuple, (which run are we using for the test data)
      norm         : string, ("RUNS": normalizing separately on each run;
                              "SUBJECT": Normalizing separately by each subject)
      returns      : nd.arrays, Concatenated X data of (time points, x*y*z) x = 79, y = 95, z = 75
                    and Concatenated y labels of (time points,)
    """

  # train and labels
  X = []
  y = []
  ##val and labels
  Xv = []
  yv = []
  # TEST and labels
  Xt = []
  yt = []
  ## decide how to concatenate runs
  if len(runs_train)>1:
      for run in runs_train:
        X.append(sub_data[run])
        y.append(sub_labels[run])
      
      X = np.concatenate(np.array(X))
      y = np.concatenate(np.array(y))
  else:
      X = sub_data[runs_train[0]-1]
      y = sub_labels[runs_train[0]-1]
  if len(runs_test)> 1:
      Xv = sub_data[runs_test[0]-1]
      yv = sub_labels[runs_test[0]-1]
      Xt = sub_data[runs_test[1]-1]
      yt = sub_labels[runs_test[1]-1]
  else:
      Xt = sub_data[runs_test[0]-1]
      yt = sub_labels[runs_test[0]-1]
  ##run standardization
  if norm == "RUNS":
      scalar = StandardScaler()
      X = scalar.fit_transform(X)
      scalarT = StandardScaler()
      Xt = scalarT.fit_transform(Xt)
      if len(Xv)>0:
        Xv = scalarT.fit_transform(Xv)
  elif norm == "SUBJECT":
      scalar = StandardScaler().fit(X)
      X = scalar.transform(X)
      

      Xt = scalar.transform(Xt)
      if len(Xv)>0:
        Xv = scalar.transform(Xv)
  else:
      print('Not doing standardization')      
  return X, y, Xt, yt, Xv, yv

In [ ]:
def run_single_subject_svm(sub_data,sub_labels,runs_train,runs_test,norm="none",do_cv=False):
  """
    Function to run cross-validation or single subject SVM
      sub_data     : (1 subject data, keys as subject ID for frmi data or labels)
      sub_labels   : sub_labels to indicate which row of the sub_data belongs to increase/decrease state
      runs_train   : tuple , (which run are we using for the training data)
                            If runs_test = 1, there will be no X_val, y_val
      runs_test    : tuple, (which run are we using for the test data)
      norm         : string, ("RUNS": normalizing separately on each run;
                              "SUBJECT": Normalizing separately by each subject
                              "none: no normalization will be done)
      do_cv:       : If True, run cross-validation
      returns      : subject individual model, Training data/labels, Val data/labels, Test data/labels
  """
  
  #get scaled data
  X_train, y_train, X_test, y_test, X_val, y_val = scale_data_single_subj(sub_data,sub_labels,runs_train,runs_test,norm)
  #run cv if do_cv = True, else run individual model SVM
  if do_cv:
    c_params = {'C':[0.7, 1, 5, 10],'kernel':['linear', 'rbf']}
    svc = SVC()
    clf = GridSearchCV(svc, c_params)
    clf.fit(X_train, y_train)
    return clf
  else:
    clf = SVC(C=10)
    clf.fit(X_train,y_train)

  return clf,X_train,y_train,X_test,y_test,X_val,y_val

In [ ]:

def run_subject_model(data_path_dict,path,runs_train,runs_test,mask_type,mask_ind,do_psc=True,norm='none'):
  """
    Function loops over subjects to grap data,scale,data,and run the SVM.
    data_path_dict  : dictionary containing paths to all data stored on AWS
    path : path to save pickle files. NOTE: If run on all subjects, this will take up about 20 GB
    runs_train: what runs do we want to train on
    runs_test: what runs do we want to test on
    mask_type: which type of brain mask do we want to apply
    mask_ind: to distinguish between ROI regions of masking(1) and full brain(or full_brain minus ROIs)(0)
    do_psc: do we want to apply Percent Signal Change normalization. Default = True,
    norm: Takes three parameters: none(no normalization will be applied),"SUBJECT"(apply normalization per subject)
          "RUNS"(apply normalization per run separately)
  """
  #get subject information
  subject_ids,subj_paths = get_subj_information(data_path_dict)
  mask_labels_indices,binary_labels = get_labels(data_path_dict)
  mask = get_mask(mask_type,data_path_dict,mask_ind)
  single_model_path={}
  for idx in range(len(subject_ids)):
    model_dict = {}
    user_data_dict, sub_id = load_subject_data(subj_paths,subject_ids,idx,do_psc,mask,mask_labels_indices,binary_labels)
    sub_data = user_data_dict[sub_id]
    sub_labels = user_data_dict[f"{sub_id}_rt_labels"]
    clf,X_train, y_train, X_test, y_test, X_val, y_val = run_single_subject_svm(sub_data,sub_labels,runs_train,runs_test,norm)
    model_dict[sub_id] = {}
    model_dict[sub_id]['model'] = clf
    model_dict[sub_id]['X_train'] = X_train
    model_dict[sub_id]['y_train'] = y_train
    model_dict[sub_id]['X_test'] = X_test
    model_dict[sub_id]['y_test'] = y_test
    if len(X_val)>0:
      model_dict[sub_id]['X_val'] = X_val
      model_dict[sub_id]['y_val'] = y_val
    destination_path = f'/content/drive/My Drive/data/model_mPFC_sub_mask/one_run_model_znorm_{sub_id}.pkl'
    filehandler = open(destination_path,"wb")
    pickle.dump(model_dict,filehandler)
    filehandler.close()
   
    # single_model_path[sub_id] = f'models/single/one_run_model_znorm_{sub_id}.pkl'
    # object_name = f"models/single/one_run_model_znorm_{sub_id}.pkl"
    # upload = s3_upload(model_dict,object_name,"pickle")
  return single_model_path,user_data_dict,model_dict

In [ ]:
file_name_dict,user_data_dict,model_dict = run_subject_model(data_path_dict,(2,),(3,4),'masksubmPFC',0,do_psc=False,norm='RUNS')


100%|██████████| 4/4 [00:01<00:00,  3.97it/s]


In [ ]:
user_data_dict['30053_10112'][0].shape

(84, 237252)

In [ ]:
def run_subject_model_cv(subject_ids, runs_train,runs_test,do_psc=True,norm='none'):
  cv_dict = {}
  for idx in range(len(subject_ids)):
    model_dict = {}
    user_data_dict, sub_id = load_subject_data(idx,do_psc)
    sub_data = user_data_dict[sub_id]
    sub_labels = user_data_dict[f"{sub_id}_rt_labels"]
    clf,val_acc,test_acc = run_single_subject_svm(sub_data,sub_labels,runs_train,runs_test,norm,do_cv=True)
    cv_dict[sub_id] = {}
    cv_dict[sub_id]['model'] = clf
    cv_dict[sub_id]['val_acc'] = val_acc
    cv_dict[sub_id]['test_acc'] = test_acc
   
  return cv_dict

In [ ]:
cv_results = run_subject_model_cv(subject_ids,(2,),(3,4),do_psc=False,norm='RUNS')